In [ ]:
# check nvidia-gpu version
!nvidia-smi

Mon Aug 31 08:49:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import google drive which contain the image file and its annotation file

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# clone the darknet repository

!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14285, done.
remote: Total 14285 (delta 0), reused 0 (delta 0), pack-reused 14285
Receiving objects: 100% (14285/14285), 12.84 MiB | 19.38 MiB/s, done.
Resolving deltas: 100% (9745/9745), done.


In [ ]:
# enable opencv, cudnn and gpu
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# compile the darknet after making change in Makefile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [ ]:

# create copy file of the yolov3 configuration file, because for custom object detection you need to make change in configuration file as per your number of custom object
# to make change create its copy file and make change in copied configuration file

!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# change in cfg file as per your requirement 

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg # change batch size =64
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg # change subdivision = 16
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg # change max_batches = 2000 * no. of custom object
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg # change at line 610 in cfg file classes = no. of custom object
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg # change at line 696 in cfg file
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg # change at line 783 in cfg file
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg # change at line 603 and set filter = (no. of custom object + 5) * 3 
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg # change at line 689
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg # change at line 776

In [ ]:
# create obj.names file which contain the name of the our cusotm object
# define multiple name at new line

!echo -e 'Person Face' > data/obj.names
# backup = '/content/gdrive/My Drive/Colab Notebooks/YOLO_facemask'
# !echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup =/backup ' > data/obj.data/


In [ ]:
# create obj.data file, which contain the location of all trainin file, testing file and no. of classes to be detect, and backup file location
# backup file location contain the location where custom weights of model will be save
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = backup' > data/obj.data


In [ ]:
# !cp 'cfg/yolov3_training.cfg' '/content/gdrive/My Drive/Colab Notebooks/YOLO_facemask/yolov3_testing.cfg'
# !cp 'data/obj.names' '/content/gdrive/My Drive/Colab Notebooks/YOLO_facemask/classes.txt'

In [ ]:
# create obj folder and unzip the image data

!mkdir /content/darknet/data/obj/
!unzip '/content/gdrive/My Drive/yolov2 facemask/images.zip' -d 'data/obj/'

Archive:  /content/gdrive/My Drive/yolov2 facemask/images.zip
  inflating: data/obj/images0.jpg    
  inflating: data/obj/images0.txt    
  inflating: data/obj/images1.jpg    
  inflating: data/obj/images1.txt    
  inflating: data/obj/images2.jpg    
  inflating: data/obj/images2.txt    
  inflating: data/obj/images3.jpg    
  inflating: data/obj/images3.txt    
  inflating: data/obj/images4.jpg    
  inflating: data/obj/images4.txt    
  inflating: data/obj/images5.jpg    
  inflating: data/obj/images5.txt    
  inflating: data/obj/images6.jpg    
  inflating: data/obj/images6.txt    
  inflating: data/obj/images7.jpg    
  inflating: data/obj/images7.txt    
  inflating: data/obj/images8.jpg    
  inflating: data/obj/images8.txt    
  inflating: data/obj/images9.jpg    
  inflating: data/obj/images9.txt    
  inflating: data/obj/images10.jpg   
  inflating: data/obj/images10.txt   
  inflating: data/obj/images11.jpg   
  inflating: data/obj/images11.txt   
  inflating: data/obj/imag

In [ ]:
# copy all .jpg images into train.txt file for training

import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
# download yolov3 pre-trained weights file
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-09-01 10:52:27--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   101KB/s    in 21m 46s 

2020-09-01 11:14:14 (121 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# train the model 
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

In [ ]:
# after 100 iteration your model custom weight automatically save into backup file location
# so, whenever you again start training of model use saved pre-trained weights instead of pre-trained weight

!./darknet detect cfg/yolov3_training.cfg '/content/darknet/backup/yolov3_training_last.weights' '/content/darknet/data/obj/0223_637269731201575895.jpg'

## **OPEN cv file for predicting on video file**

In [ ]:
import numpy as np
import cv2

net = cv2.dnn.readNet('yolov3_training_last_my.weights','yolov3_custom.cfg ')

# load the 80 class coco file
classes = []
with open('classes.names','r') as f:
    classes = f.read().splitlines()

# print(classes)

# img = cv2.imread('img.jpg')
cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    height, width, _ = img.shape


    # # preprocess the input
    blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB = True, crop = False)

    # for b in blob:
    #     for n, img_blob in enumerate(b):
    #         cv2.imshow(str(n), img_blob)
    #     print(b)

    # set the input
    net.setInput(blob)

    # pass input to forward pass
    output_layer_names = net.getUnconnectedOutLayersNames()

    # get the output from this layer
    layerOutputs = net.forward(output_layer_names)

    confidences = []
    boxes = []
    class_ids = []

    # extract the bundingbox, confidence adn class id
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]


                # scale it back to original image
            if confidence > 0.5:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                # to get the position of upper left corner
                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

                # we have more than one boxes on single object
                # to identify we choose max probability and for this we gone use NMSboxes

    print(len(boxes))
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
#     print(indexes.flatten())            

    # # pass all info and show it on img
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size = (len(boxes), 3))

    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i],2))
        color = colors[i]
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, label+ " " + confidence, (x,y+20), font, 2, (255,255,255), 3)



    cv2.imshow('img',img)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# for image detection

# correct

import numpy as np
import cv2

net = cv2.dnn.readNet('yolov3_training_last.weights','yolov3_training.cfg ')

# load the 80 class coco file
classes = []
with open('obj.names','r') as f:
    classes = f.read().splitlines()

# print(classes)

img = cv2.imread('download.jpg')
# cap = cv2.VideoCapture(0)

# while True:
# _, img = cap.read()
height, width, _ = img.shape


# # preprocess the input
blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB = True, crop = False)

# for b in blob:
#     for n, img_blob in enumerate(b):
#         cv2.imshow(str(n), img_blob)
#     print(b)

# set the input
net.setInput(blob)

# pass input to forward pass
output_layer_names = net.getUnconnectedOutLayersNames()

# get the output from this layer
layerOutputs = net.forward(output_layer_names)

confidences = []
boxes = []
class_ids = []

# extract the bundingbox, confidence adn class id
for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]


            # scale it back to original image
        if confidence > 0.5:
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)

            # to get the position of upper left corner
            x = int(center_x - w/2)
            y = int(center_y - h/2)

            boxes.append([x, y, w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

            # we have more than one boxes on single object
            # to identify we choose max probability and for this we gone use NMSboxes

print(len(boxes))
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
# print(indexes.flatten())            

# # pass all info and show it on img
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size = (len(boxes), 3))

for i in indexes.flatten():
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i],2))
    color = colors[i]
    cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
    cv2.putText(img, label+ " " + confidence, (x,y+20), font, 2, (255,255,255), 3)



cv2.imshow('img',img)
#     key = cv2.waitKey(1)
#     if key == 27:
#         break
# cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()